In [0]:
          _____                _____                    _____                    _____                    _____                    _____          
         /\    \              /\    \                  /\    \                  /\    \                  /\    \                  /\    \         
        /::\    \            /::\    \                /::\    \                /::\    \                /::\    \                /::\    \        
       /::::\    \           \:::\    \              /::::\    \              /::::\    \              /::::\    \               \:::\    \       
      /::::::\    \           \:::\    \            /::::::\    \            /::::::\    \            /::::::\    \               \:::\    \      
     /:::/\:::\    \           \:::\    \          /:::/\:::\    \          /:::/\:::\    \          /:::/\:::\    \               \:::\    \     
    /:::/__\:::\    \           \:::\    \        /:::/__\:::\    \        /:::/__\:::\    \        /:::/__\:::\    \               \:::\    \    
    \:::\   \:::\    \          /::::\    \      /::::\   \:::\    \      /::::\   \:::\    \      /::::\   \:::\    \              /::::\    \   
  ___\:::\   \:::\    \        /::::::\    \    /::::::\   \:::\    \    /::::::\   \:::\    \    /::::::\   \:::\    \    ____    /::::::\    \  
 /\   \:::\   \:::\    \      /:::/\:::\    \  /:::/\:::\   \:::\    \  /:::/\:::\   \:::\____\  /:::/\:::\   \:::\    \  /\   \  /:::/\:::\    \ 
/::\   \:::\   \:::\____\    /:::/  \:::\____\/:::/  \:::\   \:::\____\/:::/  \:::\   \:::|    |/:::/  \:::\   \:::\____\/::\   \/:::/  \:::\____\
\:::\   \:::\   \::/    /   /:::/    \::/    /\::/    \:::\  /:::/    /\::/   |::::\  /:::|____|\::/    \:::\  /:::/    /\:::\  /:::/    \::/    /
 \:::\   \:::\   \/____/   /:::/    / \/____/  \/____/ \:::\/:::/    /  \/____|:::::\/:::/    /  \/____/ \:::\/:::/    /  \:::\/:::/    / \/____/ 
  \:::\   \:::\    \      /:::/    /                    \::::::/    /         |:::::::::/    /            \::::::/    /    \::::::/    /          
   \:::\   \:::\____\    /:::/    /                      \::::/    /          |::|\::::/    /              \::::/    /      \::::/____/           
    \:::\  /:::/    /    \::/    /                       /:::/    /           |::| \::/____/               /:::/    /        \:::\    \           
     \:::\/:::/    /      \/____/                       /:::/    /            |::|  ~|                    /:::/    /          \:::\    \          
      \::::::/    /                                    /:::/    /             |::|   |                   /:::/    /            \:::\    \         
       \::::/    /                                    /:::/    /              \::|   |                  /:::/    /              \:::\____\        
        \::/    /                                     \::/    /                \:|   |                  \::/    /                \::/    /        
         \/____/                                       \/____/                  \|___|                   \/____/                  \/____/         

# Policy Iteration Exercise

## Task

Implement Policy iteration algorithm (you may use Policy Evaluation implementation from exercise 1)

**Steps1**

- Evaluate given policy. 
- Once optimal state-value function is calculated for the given policy update the policy by acting greedily with respect to state value function. The goal is to exclude "bad" actions from being executed.
- After the policy has been update, evaluate and update again until the policiy is optimal.

*Note:* you may consider policy to be optimal once it stops being updated after policy evaluation step.

## What we implement

$$
V_*(s) = max(R(s, a) + \gamma \sum_{s' \in S} P^a_{ss'} V_*(s'))
$$

## Implementation

In [27]:
# !python -m pip install -e git+https://github.com/star-ai/rl-environments.git#egg=rlenvs
# !python -m pip install gym
!pip install -e git+https://github.com/star-ai/rl-environments.git#egg=rlenvs
!pip install gym

Obtaining rlenvs from git+https://github.com/star-ai/rl-environments.git#egg=rlenvs
  Cloning https://github.com/star-ai/rl-environments.git to ./src/rlenvs
  Running setup.py develop for rlenvs
    100% |████████████████████████████████| 1.5MB 9.5MB/s 
    100% |████████████████████████████████| 1.0MB 11.6MB/s 
  Running setup.py bdist_wheel for gym ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/ea/ec/dd/33bcc8801d345f0b640fced8a0864a7c8474828564bc5ccf70
Successfully built gym


In [0]:
from IPython.core.debugger import set_trace
import numpy as np
import pprint

# Import below can all of a sudden break
# NOTE: if running locally, remove src.rlenvs from import
from src.rlenvs.rlenvs.envs.gridworld import GridworldEnv

In [0]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [0]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    # Taken from Policy Evaluation Exercise!
    
    while True:
        old_V = V.copy()
        for state in range(env.nS):
          V[state] = 0
          for action, action_prob in enumerate(policy[state]):
            for prob, next_state, reward, _ in env.P[state][action]:
              V[state] += action_prob * prob * (reward + discount_factor * old_V[next_state])
        # Stop evaluating once our value function change is below a threshold
        if np.max(np.abs(V - old_V)) < theta:
          break
    return np.array(V)

In [71]:
policy_eval(policy, env)

array([  0.        , -13.99989315, -19.99984167, -21.99982282,
       -13.99989315, -17.99986052, -19.99984273, -19.99984167,
       -19.99984167, -19.99984273, -17.99986052, -13.99989315,
       -21.99982282, -19.99984167, -13.99989315,   0.        ])

In [0]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        prev_policy = policy.copy()
        # Implement this!
        V = policy_eval_fn(policy, env)
        q = np.array([[sum([p * (r + discount_factor * V[s_]) for p, s_, r, _ in env.P[s][a]]) for a in range(env.nA)] for s in range(env.nS)])
        max_a = np.max(q, axis = 1)
        policy = np.equal(q.transpose(), max_a.transpose())
        policy = (policy / np.sum(policy, axis = 0)).transpose()
        if np.allclose(prev_policy, policy):
          break
    
    return policy, V

In [204]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution:
[[0.25 0.25 0.25 0.25]
 [0.   0.   0.   1.  ]
 [0.   0.   0.   1.  ]
 [0.   0.   0.5  0.5 ]
 [1.   0.   0.   0.  ]
 [0.5  0.   0.   0.5 ]
 [0.25 0.25 0.25 0.25]
 [0.   0.   1.   0.  ]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   0.5  0.5  0.  ]
 [0.   0.   1.   0.  ]
 [0.5  0.5  0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.25 0.25 0.25 0.25]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

